### INST447 Project Part 2
#### Members: Alexander Chui and Matthew Chan

### What factors affect ratings for an anime?

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing the required libraries
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import numpy as np
from requests import get

In [3]:
#libraries imported for handling rate limits and console formatting
from time import time, sleep
from random import randint
from warnings import warn
from IPython.core.display import clear_output

In [4]:
import regex as re

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis

import nltk; 

C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\MaNinjaChan\anaconda3\lib\site-packages\sklearn\linear_model\_least_angle.py:164: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar'

In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
#Connecting to the URL on the web to access its content (Crunchyroll)
# html=urlopen('https://www.crunchyroll.com/fullmetal-alchemist-brotherhood/reviews/helpful/page1')
# html.status

In [7]:
#Connecting to the URL on the web to access its content (MAL)
html=urlopen('https://myanimelist.net/anime/2001/Tengen_Toppa_Gurren_Lagann/reviews?p=1')
html.status

200

In [8]:
#Creating an instance of beautifulsoup object to begin web scraping
bs=BeautifulSoup(html.read(),'html.parser')

In [9]:
#Acessing the container objects with the anime info 
anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})
print(len(anime_review_containers))

20


In [10]:
#Creating empty lists to store all the items 
review = []

In [11]:
# Preparing the monitoring of the loop
start_time = time()
request = 0

In [12]:
#Extracting each job name in loop
for page in range(1,27):
    # Make a get request
        #https://myanimelist.net/anime/genre/1/Action?page=1
        #https://myanimelist.net/anime/genre/1/Action?page=2
        response = get('https://myanimelist.net/anime/2001/Tengen_Toppa_Gurren_Lagann/reviews?p='+ str(page))
        
#         # Pause the loop
#         sleep(randint(8,15))

#         # Monitor the requests
#         request += 1
#         elapsed_time = time() - start_time
#         print('Request:{}; Frequency: {} requests/s'.format(request, 
#                                                             request/elapsed_time))
#         clear_output(wait = True)

#         # Throw a warning for non-200 status codes
#         if response.status_code != 200:
#             warn('Request: {}; Status code: {}'.format(request, 
#                                                        response.status_code))

#         # Break the loop if the number of requests is greater than expected
#         if request > 100:
#             warn('Number of requests was greater than expected.')
#             break
        
        # Parse the content of the request with BeautifulSoup
        bs = BeautifulSoup(response.text, 'html.parser')
        
        #Acessing the container objects with the anime info 
        anime_review_containers = bs.find_all('div', {'class':{'borderDark'}})

        #Extracting all the elements 
        for container in anime_review_containers:
        # If the movie has Metascore, then extract:
        #  if container.find('div', {'class':{'scoremem'}})  is not None:
        
        # The name of anime
            try: 
                reviews = container.find('div', {'class':'spaceit textReadability word-break pt8 mt8'})
                review.append(reviews.text)
            except:
                review.append('Missing')

In [13]:
#Creating a pandas dataframe to store the objects extracted
anime_review = pd.DataFrame({'Review': review
})
print(anime_review.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  520 non-null    object
dtypes: object(1)
memory usage: 4.2+ KB
None


In [14]:
# Display the top 5 rows of dataset
anime_review.head()

,Review
0,\n\n\n\nOverall\n10\n\n\nStory\n10\n\n\nAnimat...
1,\n\n\n\nOverall\n7\n\n\nStory\n6\n\n\nAnimatio...
2,\n\n\n\nOverall\n10\n\n\nStory\n9\n\n\nAnimati...
3,\n\n\n\nOverall\n9\n\n\nStory\n7\n\n\nAnimatio...
4,\n\n\n\nOverall\n5\n\n\nStory\n3\n\n\nAnimatio...


In [15]:
# Convert to list
data = anime_review['Review'].tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

print(data[0])

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-15-b412c0db5fd7>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


 Overall 10 Story 10 Animation 10 Sound 10 Character 10 Enjoyment 10 There was a time. When men were men. They were screaming like crazy speeches, faced the impossible, and said things that made no sense, but that even so we did say "EPIC". There was a time. A time of captains Harlock, a master time Asia, a time of Domons. Logic was not important. The strategy was not important. If you believe in the impossible, impossible would be nothing in front of you. The time is gone A wave of animes that have to be "realistic", "philosophical", "adults" around us. Are good? Are. Meet your purpose? Meet. However, however ... I miss something. Characters idiots who laugh in the face of death, and not to be shaken by anything. Characters that I remember that, in essence, in the beginning, were not meant to be realistic. No wonder that I am addicted to shonens: manga as One Piece, Fairy Tail remind me that there are heroes who still believe that boundaries are bullshit, and that men should talk abou

In [16]:
# Clean and tokenize the reviews

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
data_words = list(sent_to_words(data))

print(data_words[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'there', 'was', 'time', 'when', 'men', 'were', 'men', 'they', 'were', 'screaming', 'like', 'crazy', 'speeches', 'faced', 'the', 'impossible', 'and', 'said', 'things', 'that', 'made', 'no', 'sense', 'but', 'that', 'even', 'so', 'we', 'did', 'say', 'epic', 'there', 'was', 'time', 'time', 'of', 'captains', 'harlock', 'master', 'time', 'asia', 'time', 'of', 'domons', 'logic', 'was', 'not', 'important', 'the', 'strategy', 'was', 'not', 'important', 'if', 'you', 'believe', 'in', 'the', 'impossible', 'impossible', 'would', 'be', 'nothing', 'in', 'front', 'of', 'you', 'the', 'time', 'is', 'gone', 'wave', 'of', 'animes', 'that', 'have', 'to', 'be', 'realistic', 'philosophical', 'adults', 'around', 'us', 'are', 'good', 'are', 'meet', 'your', 'purpose', 'meet', 'however', 'however', 'miss', 'something', 'characters', 'idiots', 'who', 'laugh', 'in', 'the', 'face', 'of', 'death', 'and', 'not', 'to', 'be', 'shaken', 'by', 'anything

In [17]:
# Create bigrams, remove stop words and lemmatize the reviews

# Build the bigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

print(bigram_mod[data_words[4]])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'gainax', 'is', 'easily', 'one', 'of', 'the', 'most', 'well', 'known', 'anime', 'studio', 'especially', 'with', 'titles', 'like', 'neon_genesis', 'evangelion', 'karekano', 'and', 'wings', 'of', 'honneamise', 'however', 'as', 'of', 'late', 'many', 'fans', 'claim', 'that', 'the', 'quality', 'of', 'gainax', 'has', 'dropped', 'down', 'significantly', 'and', 'that', 'their', 'golden', 'age', 'is', 'over', 'well', 'that', 'was', 'until', 'gurren', 'lagann', 'without_doubt', 'gurren', 'lagann', 'took', 'both', 'japanese', 'and', 'western', 'fandom', 'by', 'storm', 'this', 'coupled', 'with', 'the', 'fact', 'that', 'there', 'was', 'very', 'little', 'hype', 'before', 'hand', 'thus', 'making', 'the', 'extreme', 'popularity', 'even', 'more', 'impressive', 'for', 'me', 'what', 'was', 'more', 'impressive', 'was', 'the', 'fact', 'that', 'people', 'that', 'normally', 'don', 'rsquo', 'like', 'this', 'sort', 'of', 'thing', 'ie', 'prefe

In [18]:
# Defining removing stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Defining making bigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# Defining lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['overall', 'story', 'animation', 'sound', 'character', 'enjoyment', 'time', 'man', 'man', 'scream', 'crazy', 'speech', 'face', 'impossible', 'say', 'thing', 'make', 'sense', 'even', 'say', 'epic', 'time', 'time', 'captain', 'harlock', 'master', 'time', 'asia', 'time', 'domon', 'logic', 'important', 'strategy', 'important', 'believe', 'impossible', 'impossible', 'front', 'time', 'go', 'wave', 'anime', 'realistic', 'philosophical', 'adult', 'good', 'meet', 'purpose', 'meet', 'however', 'however', 'miss', 'character', 'idiot', 'laugh', 'face', 'death', 'shake', 'character', 'remember', 'essence', 'beginning', 'mean', 'realistic', 'wonder', 'addict', 'shonen', 'manga', 'piece', 'fairy', 'tail', 'remind', 'hero', 'still', 'believe', 'boundary', 'bullshit', 'man', 'talk', 'crazy', 'dream', 'smile', 'face', 'original', 'anime', 'non', 'manga', 'base', 'pass', 'spirit', 'indicate', 'studio', 'produce', 'epic', 'notion', 'gainax', 'probably', 'first', 'list', 'lady', 'gentleman', 'forgiveness'

In [20]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[0])

[(0, 3), (1, 1), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 12), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 6), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 7), (35, 1), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 1), (50, 2), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 5), (82, 1), (83, 7), (84, 1), (85, 2), (86, 1), (87, 2), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 2), (97, 3), (98, 5), (99, 1), (100, 1), (101, 1), (102, 1), (103, 2), (104, 1), (105, 2), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)

In [21]:
# Using LDA to develop a topic model with 3 models

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
# Print the KeywordS in the 3 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.009*"que" + 0.006*"nao" + 0.004*"com" + 0.003*"como" + 0.003*"minha" + 0.003*"tem" + 0.002*"ser" + 0.002*"ante" + 0.002*"ttgl" + 0.002*"seus"'), (1, '0.000*"show" + 0.000*"character" + 0.000*"gurren" + 0.000*"get" + 0.000*"lagann" + 0.000*"first" + 0.000*"really" + 0.000*"series" + 0.000*"feel" + 0.000*"good"'), (2, '0.032*"character" + 0.030*"anime" + 0.022*"story" + 0.021*"show" + 0.013*"watch" + 0.012*"good" + 0.012*"overall" + 0.012*"animation" + 0.012*"sound" + 0.011*"episode"'), (3, '0.017*"dude" + 0.003*"sakuga" + 0.003*"btw" + 0.002*"namely" + 0.002*"chore" + 0.001*"tease" + 0.001*"uncomfortable" + 0.001*"syndrome" + 0.001*"brutality" + 0.001*"alternative"'), (4, '0.002*"nihilism" + 0.001*"thusly" + 0.001*"slave" + 0.001*"attain" + 0.001*"meta" + 0.001*"external" + 0.001*"acceptance" + 0.001*"rationality" + 0.001*"promote" + 0.001*"bridge"'), (5, '0.017*"kamina" + 0.016*"simon" + 0.015*"gurren" + 0.015*"lagann" + 0.013*"character" + 0.012*"show" + 0.007*"series" + 0.006

In [22]:
# Generate a visualization of the topic models

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.339133  0.000182       1        1  67.257330
5      0.229997 -0.041430       2        1  28.776777
6     -0.048225  0.237927       3        1   3.413531
0     -0.121817 -0.058552       4        1   0.347644
4     -0.102241 -0.036178       5        1   0.111539
3     -0.102960 -0.037576       6        1   0.081672
7     -0.096982 -0.032218       7        1   0.006574
1     -0.096906 -0.032155       8        1   0.004933, topic_info=           Term         Freq        Total Category  logprob  loglift
8         anime  2463.000000  2463.000000  Default  30.0000  30.0000
157      kamina   803.000000   803.000000  Default  29.0000  29.0000
276       simon   752.000000   752.000000  Default  28.0000  28.0000
608       theme   206.000000   206.000000  Default  27.0000  27.0000
327       watch  1057.000000  1057.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
6      analysis     0.000673    10.291767   Topic8  -9.0878   0.2818
9    antithesis     0.000673     3.307188   Topic8  -9.0878   1.4170
10   arrogantly     0.000673     0.756414   Topic8  -9.0878   2.8923
11         asia     0.000673     0.756301   Topic8  -9.0878   2.8924
12   atmosphere     0.000673    21.807253   Topic8  -9.0878  -0.4691

[467 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
3347      1  0.710552       abhor
343       1  0.993869  absolutely
343       2  0.010142  absolutely
652       2  0.293027   absurdity
652       3  0.683729   absurdity
...     ...       ...         ...
647       1  0.394420       world
647       2  0.607871       world
8460      3  0.981328    wrapping
649       1  0.548087        yoko
649       2  0.451366        yoko

[441 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 7, 1, 5, 4, 8, 2])

In [23]:
# Save the data frame as a CSV file
# job_datas.to_csv(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Programming Assignment 2\Chan_Matthew_INST447_PA2.csv')

In [24]:
# Week 9 Topic Modelling
# ner_corpus = ' '.join(ner_df['Name'])
# corp = open(r'D:\UMD Senior College Work\Spring 2021\INST447-0101\Week 9\corpus.txt',"w")
# corp.writelines(ner_corpus)
# corp.close()